<a href="https://colab.research.google.com/github/Medhashah03/SYNAPSE_LP/blob/Week-8/Latent_Dirichlet_Allocation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Latent Dirichlet Allocation ##
**Watch these videos:**
- https://www.youtube.com/watch?v=azXCzI57Yfc
- https://www.youtube.com/watch?v=T05t-SqKArY
- https://www.youtube.com/watch?v=BaM1uiCpj_E

LDA is used to classify text in a document to a particular topic. It builds a topic per document model and words per topic model, modeled as Dirichlet distributions. 

* Each document is modeled as a multinomial distribution of topics and each topic is modeled as a multinomial distribution of words.
* LDA assumes that the every chunk of text we feed into it will contain words that are somehow related. Therefore choosing the right corpus of data is crucial. 
* It also assumes documents are produced from a mixture of topics. Those topics then generate words based on their probability distribution. 

## Step 1: Load the dataset

The dataset we'll use is a list of over one million news headlines published over a period of 15 years. We'll start by loading it from the `abcnews-date-text.csv` file.

In [ ]:
'''
Load the dataset from the CSV and save it to 'data_text'. We only need the'headline_text' column,
so drop the rest of the columns. Call the final dataframe "documents" 
Then, add an 'index' column to the dataframe. Be sure to name it index.
Hint: dataframe.index
'''
import pandas as pd
from google.colab import drive
drive.mount("/content/gdrive")
data_text = pd.read_csv("/content/gdrive/MyDrive/NLP_dataset/abcnews-date-text.csv")
documents =data_text.drop(data_text.columns.difference(['headline_text']), axis=1)
documents['index']=documents.index
documents.head()



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


Let's glance at the dataset:

In [ ]:
'''
print the total number of documents
'''
len(documents['index'])


1103665

In [ ]:
documents[:5]

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


## Step 2: Data Preprocessing ##

We will perform the following steps:

* **Tokenization**: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
* Words that have fewer than 3 characters are removed.
* All **stopwords** are removed.
* Words are **lemmatized** - words in third person are changed to first person and verbs in past and future tenses are changed into present.
* Words are **stemmed** - words are reduced to their root form.

This time, we'll make use of the gensim library and show you how  to preprocess text easily. 

In [ ]:
'''
Loading Gensim and nltk libraries
'''
# pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

In [ ]:
import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Lemmatizer Example
Before preprocessing our dataset, let's first look at an lemmatizing example. What would be the output if we lemmatized the word 'went':

In [ ]:
print(WordNetLemmatizer().lemmatize('went', pos = 'v')) # past tense to present tense

go


### Stemmer Example
Let's also look at a stemming example. Let's throw a number of words at the stemmer and see how it deals with each one:

In [ ]:
stemmer = SnowballStemmer("english")
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]

pd.DataFrame(data={'original word':original_words, 'stemmed':singles })

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [ ]:
'''
Write a function to perform the pre processing steps on the entire dataset
'''

def lemmatize_stemming(text):
    '''
        1) Lemmatize the text
        2) Then, stem the text
        3) return final text
    '''
    '''sentences = nltk.sent_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    for i in range(len(sentences)):
      words = nltk.word_tokenize(sentences[i])
      words = [stemmer.stem(word) for word in words if word not in set(stopwords.words('english'))]
      sentences[i] = ' '.join(words)
     
    for i in range(len(sentences)):
      wo = nltk.word_tokenize(sentences[i])
      wo = [lemmatizer.lemmatize(word) for word in wo if word not in set(stopwords.words('english'))]
      sentences[i] = ' '.join(words)'''

    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))



# Tokenize and lemmatize. Look up how preprocessing is being done
def preprocesser_words(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        k=len(token)
        if token not in gensim.parsing.preprocessing.STOPWORDS and (k>3):
          result.append(lemmatize_stemming(token))
            
            # TODO: Apply lemmatize_stemming on the token, then add to the results list
          
        
        
    return result


In [ ]:
'''
Preview a document after preprocessing
'''
document_num = 4310
doc_sample = documents[documents['index'] == document_num].values[0][0]

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocesser_words(doc_sample))

Original document: 
['rain', 'helps', 'dampen', 'bushfires']


Tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


Let's now preprocess all the news headlines we have. To do that, let's use the [map](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.map.html) function from pandas to apply `preprocess()` to the `headline_text` column

**Note**: This may take a few minutes

In [ ]:
# TODO: preprocess all the headlines, saving the list of results as 'processed_docs'
processed_docs = documents['headline_text'].map(preprocesser_words)

In [ ]:
'''
Preview 'processed_docs'
'''
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

## Step 3.1: Bag of words on the dataset

Now let's create a dictionary from 'processed_docs' containing the number of times a word appears in the training set. To do that, let's pass `processed_docs` to [`gensim.corpora.Dictionary()`](https://radimrehurek.com/gensim/corpora/dictionary.html) and call it '`dictionary`'.

In [ ]:
'''
Create a dictionary from 'processed_docs' containing the number of times a word appears 
in the training set using gensim.corpora.Dictionary and call it 'dictionary'
'''
#  Todo
dictionary = gensim.corpora.Dictionary(processed_docs)

In [ ]:
'''
Checking dictionary created
'''
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


** Gensim filter_extremes **

[`filter_extremes(no_below=5, no_above=0.5, keep_n=100000)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.filter_extremes)

Filter out tokens that appear in

* less than no_below documents (absolute number) or
* more than no_above documents (fraction of total corpus size, not absolute number).
* after (1) and (2), keep only the first keep_n most frequent tokens (or keep all if None).

In [ ]:
'''
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 10% of all documents
- keep 100000 most frequent tokens
'''
# TODO: apply dictionary.filter_extremes() with the parameters mentioned above
dictionary.filter_extremes(no_below=15, no_above=0.1,keep_n=100000)

** Gensim doc2bow **

[`doc2bow(document)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2bow)

* Convert document (a list of words) into the bag-of-words format = list of (token_id, token_count) 2-tuples. Each word is assumed to be a tokenized and normalized string (either unicode or utf8-encoded). No further preprocessing is done on the words in document; apply tokenization, stemming etc. before calling this method.

In [ ]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'.
Hint: Since we want a BoW model for each document, you can use list comprehension instead of for loops. List
comprehension is the more pythonic way.
'''
#  Todo:
bow_corpus = [dictionary.doc2bow(document) for document in processed_docs]

In [ ]:
'''
Checking Bag of Words corpus for our sample document --> (token_id, token_count)
'''
bow_corpus[document_num]

[(76, 1), (112, 1), (483, 1), (4014, 1)]

In [ ]:
'''
Preview BOW for our sample preprocessed document
'''
# Here document_num is document number 4310 which we have checked in Step 2
bow_doc_4310 = bow_corpus[document_num]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 76 ("bushfir") appears 1 time.
Word 112 ("help") appears 1 time.
Word 483 ("rain") appears 1 time.
Word 4014 ("dampen") appears 1 time.


## Step 3.2: TF-IDF on our document set ##

While performing TF-IDF on the corpus is not necessary for LDA implemention using the gensim model, it is recemmended. TF-IDF expects a bag-of-words (integer values) training corpus during initialization. During transformation, it will take a vector and return another vector of the same dimensionality.

*Please note: The author of Gensim dictates the standard procedure for LDA to be using the Bag of Words model.*

** TF-IDF stands for "Term Frequency, Inverse Document Frequency".**

* It is a way to score the importance of words (or "terms") in a document based on how frequently they appear across multiple documents.
* If a word appears frequently in a document, it's important. Give the word a high score. But if a word appears in many documents, it's not a unique identifier. Give the word a low score.
* Therefore, common words like "the" and "for", which appear in many documents, will be scaled down. Words that appear frequently in a single document will be scaled up.

In other words:

* TF(w) = `(Number of times term w appears in a document) / (Total number of terms in the document)`.
* IDF(w) = `log_e(Total number of documents / Number of documents with term w in it)`.

** For example **

* Consider a document containing `100` words wherein the word 'tiger' appears 3 times. 
* The term frequency (i.e., tf) for 'tiger' is then: 
    - `TF = (3 / 100) = 0.03`. 

* Now, assume we have `10 million` documents and the word 'tiger' appears in `1000` of these. Then, the inverse document frequency (i.e., idf) is calculated as:
    - `IDF = log(10,000,000 / 1,000) = 4`. 

* Thus, the Tf-idf weight is the product of these quantities: 
    - `TF-IDF = 0.03 * 4 = 0.12`.

In [ ]:
'''
Create tf-idf model object using models.TfidfModel on 'bow_corpus' and save it to 'tfidf'
'''
from gensim import corpora, models
# TODO
tf_idf = models.TfidfModel(bow_corpus)

In [ ]:
'''
Apply transformation to the entire corpus and call it 'corpus_tfidf'
'''
# TODO
corpus_tfidf = tf_idf[bow_corpus]

In [ ]:
'''
Preview TF-IDF scores for our first document --> --> (token_id, tfidf score)
'''
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5892908644709983),
 (1, 0.38929657403503015),
 (2, 0.4964985198530063),
 (3, 0.5046520328695662)]


## Step 4.1: Running LDA using Bag of Words ##

We are going for 10 topics in the document corpus.

** We will be running LDA using all CPU cores to parallelize and speed up model training.**

Some of the parameters we will be tweaking are:

* **num_topics** is the number of requested latent topics to be extracted from the training corpus.
* **id2word** is a mapping from word ids (integers) to words (strings). It is used to determine the vocabulary size, as well as for debugging and topic printing.
* **workers** is the number of extra processes to use for parallelization. Uses all available cores by default.
* **alpha** and **eta** are hyperparameters that affect sparsity of the document-topic (theta) and topic-word (lambda) distributions. We will let these be the default values for now(default value is `1/num_topics`)
    - Alpha is the per document topic distribution.
        * High alpha: Every document has a mixture of all topics(documents appear similar to each other).
        * Low alpha: Every document has a mixture of very few topics

    - Eta is the per topic word distribution.
        * High eta: Each topic has a mixture of most words(topics appear similar to each other).
        * Low eta: Each topic has a mixture of few words.

* ** passes ** is the number of training passes through the corpus. For  example, if the training corpus has 50,000 documents, chunksize is  10,000, passes is 2, then online training is done in 10 updates: 
    * `#1 documents 0-9,999 `
    * `#2 documents 10,000-19,999 `
    * `#3 documents 20,000-29,999 `
    * `#4 documents 30,000-39,999 `
    * `#5 documents 40,000-49,999 `
    * `#6 documents 0-9,999 `
    * `#7 documents 10,000-19,999 `
    * `#8 documents 20,000-29,999 `
    * `#9 documents 30,000-39,999 `
    * `#10 documents 40,000-49,999` 

In [ ]:
# LDA multicore 
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'

If Multicore is throwing errors on you laptop, use LDA mono-core, gensim.models.LdaModel
'''
# TODO
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2)


In [ ]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

Topic: 0 
Words: 0.030*"polic" + 0.028*"charg" + 0.028*"court" + 0.021*"murder" + 0.017*"face" + 0.017*"alleg" + 0.014*"year" + 0.014*"jail" + 0.014*"accus" + 0.013*"child"


Topic: 1 
Words: 0.043*"trump" + 0.029*"govern" + 0.017*"help" + 0.012*"fund" + 0.012*"work" + 0.012*"say" + 0.012*"plan" + 0.011*"water" + 0.011*"servic" + 0.011*"communiti"


Topic: 2 
Words: 0.025*"death" + 0.021*"donald" + 0.018*"attack" + 0.018*"polic" + 0.018*"brisban" + 0.017*"kill" + 0.017*"sydney" + 0.016*"melbourn" + 0.015*"china" + 0.015*"miss"


Topic: 3 
Words: 0.022*"die" + 0.015*"take" + 0.014*"korea" + 0.014*"minist" + 0.012*"prison" + 0.011*"drum" + 0.011*"find" + 0.011*"energi" + 0.011*"releas" + 0.011*"hold"


Topic: 4 
Words: 0.019*"open" + 0.016*"tasmanian" + 0.014*"perth" + 0.014*"australian" + 0.013*"break" + 0.012*"countri" + 0.011*"game" + 0.010*"young" + 0.010*"deal" + 0.009*"make"


Topic: 5 
Words: 0.027*"north" + 0.024*"nation" + 0.021*"coast" + 0.018*"power" + 0.016*"rise" + 0.015*"ca

### Classification of the topics ###

Using the words in each topic and their corresponding weights, what categories were you able to infer?

* 0: election
* 1: match scores related
* 2: casualties related
* 3: price
* 4: legal actions
* 5: service
* 6: 
* 7:  
* 8: 
* 9: 

## Step 4.2 Running LDA using TF-IDF ##

In [ ]:
'''
Define lda model using corpus_tfidf
'''
# TODO
lda_model_tfidf = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2)

In [ ]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

Topic: 0 Word: 0.023*"perth" + 0.021*"help" + 0.016*"tasmania" + 0.015*"chang" + 0.014*"break" + 0.014*"open" + 0.013*"marriag" + 0.013*"street" + 0.012*"busi" + 0.011*"farmer"


Topic: 1 Word: 0.028*"north" + 0.022*"coast" + 0.019*"market" + 0.015*"west" + 0.014*"share" + 0.014*"bank" + 0.014*"gold" + 0.012*"flood" + 0.011*"liber" + 0.011*"lead"


Topic: 2 Word: 0.026*"govern" + 0.024*"queensland" + 0.018*"nation" + 0.014*"plan" + 0.013*"school" + 0.013*"health" + 0.013*"indigen" + 0.012*"rise" + 0.012*"council" + 0.011*"fund"


Topic: 3 Word: 0.052*"australia" + 0.043*"trump" + 0.015*"live" + 0.015*"south" + 0.013*"test" + 0.012*"world" + 0.012*"record" + 0.011*"show" + 0.011*"korea" + 0.011*"china"


Topic: 4 Word: 0.054*"australian" + 0.017*"time" + 0.014*"polit" + 0.013*"feder" + 0.013*"royal" + 0.013*"famili" + 0.013*"vote" + 0.011*"commiss" + 0.010*"ash" + 0.009*"hit"


Topic: 5 Word: 0.029*"charg" + 0.029*"court" + 0.022*"murder" + 0.018*"face" + 0.018*"alleg" + 0.016*"polic" +

### Classification of the topics ###

As we can see, when using tf-idf, heavier weights are given to words that are not as frequent which results in nouns being factored in. That makes it harder to figure out the categories as nouns can be hard to categorize. This goes to show that the models we apply depend on the type of corpus of text we are dealing with. 

Using the words in each topic and their corresponding weights, what categories could you find?

* 0: 
* 1:  
* 2: 
* 3: 
* 4:  
* 5: 
* 6: 
* 7: 
* 8: 
* 9: 

## Step 5.1: Performance evaluation by classifying sample document using LDA Bag of Words model

We will check to see where our test document would be classified. 

In [ ]:
'''
Text of sample document 4310
'''
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [ ]:
'''
Check which topic our test document belongs to using the LDA Bag of Words model.
'''

# Our test document is document number 4310
for index, score in sorted(lda_model[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.22000007331371307	 
Topic: 0.027*"north" + 0.024*"nation" + 0.021*"coast" + 0.018*"power" + 0.016*"rise" + 0.015*"call" + 0.015*"island" + 0.015*"west" + 0.014*"price" + 0.013*"gold"

Score: 0.22000007331371307	 
Topic: 0.028*"queensland" + 0.025*"world" + 0.021*"south" + 0.020*"canberra" + 0.019*"test" + 0.017*"australia" + 0.016*"turnbul" + 0.016*"final" + 0.014*"peopl" + 0.012*"leagu"

Score: 0.22000005841255188	 
Topic: 0.043*"trump" + 0.029*"govern" + 0.017*"help" + 0.012*"fund" + 0.012*"work" + 0.012*"say" + 0.012*"plan" + 0.011*"water" + 0.011*"servic" + 0.011*"communiti"

Score: 0.21999981999397278	 
Topic: 0.022*"die" + 0.015*"take" + 0.014*"korea" + 0.014*"minist" + 0.012*"prison" + 0.011*"drum" + 0.011*"find" + 0.011*"energi" + 0.011*"releas" + 0.011*"hold"

Score: 0.019999999552965164	 
Topic: 0.030*"polic" + 0.028*"charg" + 0.028*"court" + 0.021*"murder" + 0.017*"face" + 0.017*"alleg" + 0.014*"year" + 0.014*"jail" + 0.014*"accus" + 0.013*"child"

Score: 0.0199999

## Step 5.2: Performance evaluation by classifying sample document using LDA TF-IDF model

In [ ]:
'''
Check which topic our test document belongs to using the LDA TF-IDF model.
Code would be similar to above
'''
# Todo
for index, score in sorted(lda_model_tfidf[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.6200000047683716	 
Topic: 0.023*"perth" + 0.021*"help" + 0.016*"tasmania" + 0.015*"chang" + 0.014*"break" + 0.014*"open" + 0.013*"marriag" + 0.013*"street" + 0.012*"busi" + 0.011*"farmer"

Score: 0.2199999988079071	 
Topic: 0.028*"north" + 0.022*"coast" + 0.019*"market" + 0.015*"west" + 0.014*"share" + 0.014*"bank" + 0.014*"gold" + 0.012*"flood" + 0.011*"liber" + 0.011*"lead"

Score: 0.019999999552965164	 
Topic: 0.026*"govern" + 0.024*"queensland" + 0.018*"nation" + 0.014*"plan" + 0.013*"school" + 0.013*"health" + 0.013*"indigen" + 0.012*"rise" + 0.012*"council" + 0.011*"fund"

Score: 0.019999999552965164	 
Topic: 0.052*"australia" + 0.043*"trump" + 0.015*"live" + 0.015*"south" + 0.013*"test" + 0.012*"world" + 0.012*"record" + 0.011*"show" + 0.011*"korea" + 0.011*"china"

Score: 0.019999999552965164	 
Topic: 0.054*"australian" + 0.017*"time" + 0.014*"polit" + 0.013*"feder" + 0.013*"royal" + 0.013*"famili" + 0.013*"vote" + 0.011*"commiss" + 0.010*"ash" + 0.009*"hit"

Score: 0

## Step 6: Testing model on unseen document ##

In [ ]:
unseen_document = "My favorite sports activities are running and swimming."

# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocesser_words(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.4199984669685364	 Topic: 0.030*"polic" + 0.028*"charg" + 0.028*"court" + 0.021*"murder" + 0.017*"face"
Score: 0.2199999988079071	 Topic: 0.028*"queensland" + 0.025*"world" + 0.021*"south" + 0.020*"canberra" + 0.019*"test"
Score: 0.2199999839067459	 Topic: 0.031*"australia" + 0.026*"adelaid" + 0.017*"dead" + 0.016*"tasmania" + 0.014*"victoria"
Score: 0.02000158466398716	 Topic: 0.043*"trump" + 0.029*"govern" + 0.017*"help" + 0.012*"fund" + 0.012*"work"
Score: 0.019999999552965164	 Topic: 0.025*"death" + 0.021*"donald" + 0.018*"attack" + 0.018*"polic" + 0.018*"brisban"
Score: 0.019999999552965164	 Topic: 0.022*"die" + 0.015*"take" + 0.014*"korea" + 0.014*"minist" + 0.012*"prison"
Score: 0.019999999552965164	 Topic: 0.019*"open" + 0.016*"tasmanian" + 0.014*"perth" + 0.014*"australian" + 0.013*"break"
Score: 0.019999999552965164	 Topic: 0.027*"north" + 0.024*"nation" + 0.021*"coast" + 0.018*"power" + 0.016*"rise"
Score: 0.019999999552965164	 Topic: 0.022*"elect" + 0.022*"hous" + 0

### Now, look up different ways we can transform our words or use different techniques or improvements that we can make to make our Topic Classifier more accurate. Write them down below:

**We can:**
- Double click here to append points

- 